**ERP analysis in EEG data recorded with Muse 2 using the Sibley platform**

Provides an basic way to inspect the ERP results in one EEG session <br>
[Updated: 2022-05-02]

In [ ]:
# If needed, mount Google drive
from google.colab import drive
drive.mount('/content/drive')

In [84]:
# Install software and load packages
!pip install mne
import numpy as np
import mne
import pandas as pd
from mne import Epochs
import os
import json
import sys

In [ ]:
# Path to the folder containing the session's data
# The files EEG.csv and session_info.json must be present
session = '/content/drive/MyDrive/sibley_output/20220429-142340_default_grey_bear'

found_eeg = os.path.exists(session + '/' + 'EEG.csv')
found_session_info = os.path.exists(session + '/' + 'session_info.json')
if found_eeg and found_session_info:
  print('Session:\t' + session)
else:
  print('ERROR: input file(s) not found')
  sys.exit()
session_info = json.load(open(session + '/' + 'session_info.json'))
print('Description:\t' + session_info['params']['session_type'])
eeg_device = session_info['params']['eeg_device']
print('EEG device:\t' + eeg_device)
if not session_info['params']['eeg_device'] in ['Muse 2', 'Muse S']:
  print('ERROR: the data was recorded with ' + session_info['params']['eeg_device'])
  sys.exit()
markers = session_info['EEG_marks']['EEG_markers']
markers_stim = [curr for curr in markers if curr['type']=='stimulus']
if len(markers_stim)==0:
  print('ERROR: no stim markers were found')
  sys.exit()
markers_msg = [x['name'] + '=' + str(x['mark']) for x in markers_stim]
print('Stim markers:\t' + ', '.join(markers_msg))

In [ ]:
# Load data and build basic diagnostic plots

fname = os.path.join(session, 'EEG.csv')
data = pd.read_csv(fname)
raw_eeg = data.copy()
if 'timestamps' in raw_eeg.columns:
    del raw_eeg['timestamps']
if 'marker' in raw_eeg.columns:
    del raw_eeg['marker']
ch_names = list(raw_eeg.columns)
raw_eeg = raw_eeg.transpose()
ch_types = ['eeg'] * 4
sfreq = 256  # Hz
# Create the info structure needed by MNE
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage('standard_1020')
raw = mne.io.RawArray(raw_eeg, info)
raw.filter(l_freq=0.5, h_freq=45)
raw.plot(scalings=100)
raw.plot_psd(average=False)
info_stim = mne.create_info(['STI'], sfreq, ['stim'])
data_stim = np.array(data['marker'])
t = data[['marker']].to_numpy().transpose()
raw_stim = mne.io.RawArray(t, info_stim)
raw.add_channels([raw_stim], force_update_info=True)
events = mne.find_events(raw, stim_channel='STI')

In [ ]:
# select the channels to plot
# One or more from the following list: ['TP9', 'AF7', 'AF8', 'TP10']
plot_channels = ['TP9']

for dict_stim in markers_stim:
  #print(dict_stim['name'] + '...' + str(dict_stim['mark']))
  for ch_curr in plot_channels:
    epochs = Epochs(raw, events, event_id=dict_stim['mark'], tmin=-0.25, tmax=1.25, baseline=None)
    str_title = ' '.join([session_info['params']['session_type'], ch_curr, dict_stim['name']])
    epochs.plot_image(picks=[ch_curr], title=str_title)
